# Root detection

In this notebook I will test different masking techniques to make image segmentation so that roots can be separated from the background.

### Reset notebook

In [1]:
#%reset

### Import libraries

In [2]:
from matplotlib import pyplot as plt
from skimage import exposure, measure
import numpy as np
import cv2

### Function grab_contrours

In [3]:
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

### Read image, resize and turn to gray scale

In [4]:
def scale_crop_gray(im):
    # Set image size to 1485x1050. Orig size 4676 × 3306.
    # We must use fixed size, because we can not know size of 
    # the pics taken in the future and size is used in parameter tuning.
    
    # Size 2970x2100 2min
    # Size 1485x1050 5sec
    width = int(1485)
    height = int(1050)
    dim = (width, height)
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    
    # Cut out 50 border pixels from each side
    im = im[50:, 50:]
    im = im[:-50, :-50]
    
    # Convert the image to grayscale
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    # Return color and grayscale images
    return im, im_gray

### Read images

Read two images. Uncomment two images you want to use.

In [5]:
# Read color image
#image = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.09.15_033029_060_DYY.jpg', 1)

# Hyde scanner1 example
im1 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.22_033029_036_DYY.jpg', 1)
im2 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.23_033029_037_DYY.jpg', 1)

# Hyde scanner2 example
#im1 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.16_033029_030_DYY.jpg', 1)
#im1 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.17_033029_031_DYY.jpg', 1)
#im2 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.18_033029_032_DYY.jpg', 1)

# Varrio_Scanner2 example
#im1 = cv2.imread('data/Varrio_Scanner2/VS2_T001_L001_2019.07.24_090749_132_DYY.jpg', 1)
#im2 = cv2.imread('data/Varrio_Scanner2/VS2_T001_L001_2019.07.25_091204_133_DYY.jpg', 1)

# Varrio Scanner3 example pairs
#im1 = cv2.imread('data/Varrio_Scanner3/VS3_T001_L001_2018.08.05_033133_140_DYY.jpg', 1)
#im2 = cv2.imread('data/Varrio_Scanner3/VS3_T001_L001_2018.08.06_033133_141_DYY.jpg', 1)

# Normalize contrast
im1 = cv2.normalize(im1, im1, 0, 255, cv2.NORM_MINMAX)
im2 = cv2.normalize(im2, im2, 0, 255, cv2.NORM_MINMAX)
    
# Get rescaled and cropped images in color and gray
im1, im1_gray = scale_crop_gray(im1)
im2, im2_gray = scale_crop_gray(im2)

### Make light areas mask

Filtering based on color. This is used as a prefiltering. The filtering find light areas that can be root or anything else. Edges are sharp, because no blurring is used, which would affect to the location of the root edges.

After making light area filtering, we will remove isolated pixels that are smaller than a thershold.

Input: one color image

In [6]:
def light_areas_mask_color(im):
    
    # Filter image by pixel values
    mask = cv2.inRange(im, (112, 168,152),(254, 255,254))

    # Remove isolated pixels
    mask = remove_isolated_pixels(mask, min_size=40, max_size=50000)
    
    # "Make areas fatter by 1 pixel (join roots where is holes in edges)
    mask = cv2.dilate(mask, None, iterations=1)
    
    # Return results
    return mask

### Make edge areas mask (may be useless)

Filtering based on edges

Input: one grayscale image

In [7]:
def edge_areas_mask(im_gray):
    #thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
    #                               cv2.THRESH_BINARY_INV, 11, 7)
    
    # Blur with bilateralFilter. BilateralFilter does not blur edges.
    blur = cv2.bilateralFilter(im_gray, 50, 15, 10)
    
    # Remove noise with gaussian blur
    blur = cv2.GaussianBlur(blur,(5,5),0)
    
    # Find edges with canny edge detection
    edges = cv2.Canny(blur, 50, 250)
    
    # "Make areas fatter" to fill missing pixels in root edges
    edges = cv2.dilate(edges, None, iterations=2)
    
    # Get contrours
    cnts = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    
    # Loop over our contours. Remove controus with area less than xxx.
    contours=[]
    for c in cnts:
        size = cv2.contourArea(c)
        if size > 20:
            contours.append(c)
    
    # Make empty picture
    empty = np.zeros(im_gray.shape, dtype = "uint8")
    
    # Save contours to empty image
    contours = cv2.drawContours(empty, contours, -1, 255, -1)
    
    # Remove isolated pixels
    # "Make areas thinner"
    #change = cv2.erode(change, None, iterations=1)
    # "Make areas fatter" to fill missing pixels in root edges
    contours = cv2.dilate(contours, None, iterations=1)
    
    #Return contour areas
    return contours

### Make mask that finds differences between two images

Filtering based on difference between two images

Input: two color images

In [8]:
def change_areas_mask(im1_color, im2_color):
    
    # Blur images
    im1_color = cv2.bilateralFilter(im1_color, d=3, 
                                    sigmaColor=15, sigmaSpace=10)
    im2_color = cv2.bilateralFilter(im2_color, d=3, 
                                    sigmaColor=15, sigmaSpace=10)
    
    # Calculate difference between images
    diff = cv2.absdiff(im1_color, im2_color)
    
    # Save temp image
    #cv2.imwrite('DIFFERENCE.png', diff)
    
    # Filter difference image by pixel values
   # mask = cv2.inRange(diff, (10, 1, 20), (95, 85 ,90))
    mask = cv2.inRange(diff, (10, 7, 17), (165, 170, 155))
    
    # Remove isolated pixels
    mask = remove_isolated_pixels(mask, min_size=15, max_size=5000)
    
    # "Make areas fatter" to join areas
    mask = cv2.dilate(mask, None, iterations=5)
    
    # Remove isolated pixels
    # "Make areas thinner"
    #change = cv2.erode(change, None, iterations=1)
    

    # Return results
    return mask

### Make circle around the found areas

Input: mask and processed image with only root tips

In [9]:
def make_circles_around(image, mask):
    # find the contours in the mask, then sort them from left to right
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, 
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    
    # Sort contours from left to right
    cnts = sort_contours(cnts)[0]
    
    # loop over the contours
    for (i, c) in enumerate(cnts):
    # draw the bright spot on the image
        (x, y, w, h) = cv2.boundingRect(c)
        ((cX, cY), radius) = cv2.minEnclosingCircle(c)
        cv2.circle(image, (int(cX), int(cY)), int(radius), (0, 255, 0), 2)
        #cv2.putText(image, "#{}".format(i + 1), (x, y - 15), 
        #            cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, 
        #            color=(0, 255, 0), thickness=2)
        
    # Return image
    return image, i

### Sort contours by location

Part of the function make_circles_around

In [10]:
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return cnts, boundingBoxes

### Remove isolated pixels that have size less than thershold

In [11]:
def remove_isolated_pixels(thresh, min_size, max_size):
    # Perform a connected component analysis on the thresholded
    # image and remove components that have size less than thershold
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if (numPixels > min_size) and (numPixels < max_size):
            mask = cv2.add(mask, labelMask)
            
    # Return cleaned mask
    return mask

### Add text to image

In [12]:
def add_text(im, text):
    # Set text font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Add text
    cv2.putText(img=im, text=text, org=(50,900), fontFace=font, fontScale=2, 
                color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    
    # Return result
    return im

### Print light and edge areas as binary masks

In [13]:
# Get areas
change_mask = change_areas_mask(im1, im2)
#light_mask = light_areas_mask(im2_gray)
light_mask = light_areas_mask_color(im2)
edge_mask = edge_areas_mask(im2_gray)


# Make supermask. Combine all masks.
combined_mask = light_mask & change_mask

# Remove noise from combined mask
combined_mask = remove_isolated_pixels(combined_mask, min_size=40, max_size=5000)

# From combined mask save only edges
combined_mask_edges = cv2.Canny(combined_mask, 50, 250)

# Concat gray, light_areas, edge_areas and change_areas to one image
concat1 = np.concatenate((im2_gray, light_mask), axis=0)
concat2 = np.concatenate((edge_mask, change_mask), axis=0)
concat3 = np.concatenate((concat1, concat2), axis=0)
concat4 = np.concatenate((concat3, combined_mask_edges), axis=0)

# Save combined edges as own image for Olgas tests
cv2.imwrite('edges_image.png', combined_mask_edges)

# Save image
#cv2.imwrite('Detect_roots_V2_binary.png', concat4)

True

### Print images with colors

In [14]:
# Mask orig color image with mask (Add colors to images)
light_areas = cv2.bitwise_or(im2, im2, mask=light_mask)
#edge_areas = cv2.bitwise_or(im2, im2, mask=edge_mask)
change_areas = cv2.bitwise_or(im2, im2, mask=change_mask)
combined_areas = cv2.bitwise_or(im2, im2, mask=combined_mask)

# Add circles around the roots and get number of contours (detected roots)
im2, nbr_cnts = make_circles_around(im2, combined_mask)
light_areas, nbr_cnts = make_circles_around(light_areas, combined_mask)
#edge_areas, nbr_cnts = make_circles_around(edge_areas, combined_mask)
change_areas, nbr_cnts = make_circles_around(change_areas, combined_mask)
combined_areas, nbr_cnts = make_circles_around(combined_areas, combined_mask)

# Add names to images
im2 = add_text(im2, "Original image. Number of roots :{}".format(nbr_cnts))
light_areas = add_text(light_areas, "Color filter")
#edge_areas = add_text(edge_areas, "Edge filter")
change_areas = add_text(change_areas, "Change filter (pic2-pic1)")
combined_areas = add_text(combined_areas, "Combined filter")

# Concat original, light_areas, edge_areas and change_areas to one image
concat1 = np.concatenate((im2, light_areas), axis=0)
#concat2 = np.concatenate((concat1, edge_areas), axis=0)
concat2 = np.concatenate((concat1, change_areas), axis=0)
concat3 = np.concatenate((concat2, combined_areas), axis=0)

# Save images
cv2.imwrite('Detect_roots_V2_color.png', concat3)
cv2.imwrite('Detected_growth.png', im2)

True